# Imports


In [ ]:
import numpy as np
import cmath
import matplotlib.pyplot as plt
import matplotlib.image as mpimg 
from scipy.fft import dct, idct

# Imported Classes

In [ ]:
class inner_product_2D(): 
  # 2D inner product for two signals of length N
  def __init__(self, x, y):
    self.x = x
    self.y = y
    self.N = len(x)

  def solve(self): 
    # 2D inner product is x(n,m)y*(n,m)
    inner_product = 0
    for i in range (self.N):
      for j in range (self.N):
        inner_product = inner_product + self.x[i,j] * np.conj(self.y[i,j])
    
    return inner_product

class comp_exp_2D(): 
  # 2D comp exp is exp(j2pi(kn/N + lm/M))
  def __init__(self, k, l, N): 
    self.k = k
    self.l = l
    self.N = N

  def solve(self): 
    comp_exp_kl = np.zeros([self.N, self.N], dtype = np.complex)
    for i in range(self.N): 
      for j in range(self.N):
        comp_exp_kl[i,j] = np.exp(-1j*2*cmath.pi*self.k*i/self.N) * np.exp(-1j*2*cmath.pi*self.l*m*self.N)
  
    comp_exp_real = np.real(comp_exp_kl)
    comp_exp_imag = np.imag(comp_exp_kl)
    return comp_exp_kl, comp_exp_real, comp_exp_imag

class square_pulse_2D(): 
  def __init__(self, N, L):
    self.N = N
    self.L = L
    self.samples = N*N

  def solve(self):
    square_pulse = np.zeros([self.N, self.N], dtype = np.float)
    for i in range (self.L):
      for j in range (self.L):
        square_pulse[i][j] = 1 / (self.L * self.L)
    
    return square_pulse, self.samples 

class Gaussian_2D():
  def __init__ (self, N, mu, sigma):
    self.N = N
    self.sigma = sigma 
    self.mu = mu
    self.samples = N*N

  def solve(self):
    gaussian = np.zeros([self.N, self.N], dtype=np.float)    #Gaussian pulse exclusively real 
    for i in range(self.N):
      for j in range(self.N):
        gaussian[i][j] = np.exp(-1*((i-self.mu)**2 + (j-self.mu)**2)/(2*(self.sigma)**2))
    
    return gaussian, self.samples 

class DFT_2D():
  def __init__(self, x):
    self.x = x
    self.M = np.shape(x)[0]
    self.N = np.shape(x)[1]
  
  def solve(self):
    DFT = np.zeros([self.M, self.N], dtype=np.complex)
    for m in range(self.M):
      for n in range(self.N):
        for i in range(self.M):
          for j in range(self.N):
            DFT[m][n] = DFT[m][n] + self.x[i][j]/np.sqrt(self.M*self.N) * np.exp(-1j*2*cmath.pi*(m*i/self.M + n*j/self.N))
    return DFT

  def solve2(self):
    X = np.fft.fft2(self.x) / np.sqrt(self.M * self.N)
    X_shift = np.fft.fftshift(X)
    return X_shift

class iDFT_2D():
  def __init__(self, X):
    self.X = X
    self.M = np.shape(X)[0]
    self.N = np.shape(X)[1]
  
  def solve(self):
    iDFT = np.zeros([self.M, self.N], dtype=np.complex)
    for m in range(self.M):
      for n in range(self.N):
        for i in range(int(self.M/2)+1):
          for j in range(self.N):
            iDFT[m][n] = iDFT[m][n] + self.X[i][j]/np.sqrt(self.M*self.N) * np.exp(1j*2*cmath.pi*(m*i/self.M + n*j/self.N))
            if i != 0:
              iDFT[m][n] = iDFT[m][n] + np.conj(self.X[i][j])/np.sqrt(self.M*self.N) * np.exp(1j*2*cmath.pi*(-m*i/self.M - n*j/self.N))
    return iDFT

class iDCT_2D():
  def __init__(self, X):
    self.X = X
    
  def solve(self):
    # use built-in Python function 
    return idct(idct(self.X.T, norm = 'ortho').T, norm='ortho')

# Section 1.1


In [ ]:
class dis_cos_2D(object):
  def __init__(self, k, l, M, N):
    self.k = k
    self.l = l
    self.M = M
    self.N = N
    self.n = np.arange(N)
    self.m = np.arange(M)
    self.dis_cos_2d = np.matmul( np.cos(self.k * cmath.pi / 2/self.M * (2*self.m+1) ).reshape((self.M,1)) ,np.cos(self.l * cmath.pi / 2/self.N * (2*self.n+1)).reshape((1,self.N)) )

#2D Discrete Cosine Transform
class DCT_2D():
  def __init__(self, x):
    self.x = x
    self.M = np.shape(x)[0]
    self.N = np.shape(x)[1]

  # Written DCT function (slower)
  def solve(self):
    DCT = np.zeros([self.M, self.N])
    for i in range(self.M):
      if i == 0:
        c1 = 1 / np.sqrt(2)
      else:
        c1 = 1
      for j in range(self.N):
        if j == 0:
          c2 = 1 / np.sqrt(2)
        else:
          c2 = 1

        dis_cos_2d = dis_cos_2D(i, j, self.M, self.N).dis_cos_2d
        inner_product = inner_product_2D(self.x, dis_cos_2d)
        DCT[i, j] = 2 / np.sqrt(self.M) / np.sqrt(self.N) * c1 * c2 * inner_product.solve()
    return DCT


  # Built-In DCT function from scipy
  def solve2(self):
    return dct(dct(self.x.T, norm='ortho').T, norm='ortho')

# Section 1.2

In [ ]:
""" class that takes in a signal (image) of size N2 and partitions it into patches 
of size 8 × 8, and for each patch stores the K largest DFT coefficients and 
their associated frequencies. We still want the patch to be 8 × 8 so set the 
other 64 − K coefficients to zero. """ 

def compress_block(X, K):
  x_trunc = np.zeros([8, 8], dtype=np.complex)
  e = np.zeros(64)
  e = abs(X.reshape((1, 64)))
  index_track = np.argsort(-e)
  index = np.array(index_track[:, 0:K])[0]
  for k in np.arange(len(index)):
    index_x = np.int(np.floor(index[k] / 8))
    index_y = np.int(index[k] - index_x * 8)
    x_trunc[index_x, index_y] = X[index_x, index_y]
  return x_trunc, index

def compress_img_DCT_2D(x, K):
  N = x.shape[0]
  compressed = np.zeros([N,N])
  for k in np.arange(0, N+1-8, 8):
    for l in np.arange(0, N+1-8, 8):
      block = x[k:k+8, l:l+8] # 8 x 8 block
      DCT_block = DCT_2D(block) # create DCT object
      DCT = DCT_block.solve2() # solve for DCT
      DCT_trunc, index = compress_block(DCT, K) # compress
      compressed[k:k+8, l:l+8] = DCT_trunc
  return compressed

def compress_img_DFT_2D(x, K):
  N = x.shape[0]
  compressed = np.zeros([N,N])
  for k in np.arange(0, N+1-8, 8):
    for l in np.arange(0, N+1-8, 8):
      block = x[k:k+8, l:l+8] # 8 x 8 block
      DFT_block = DFT_2D(block) # create DCT object
      DFT = DFT_block.solve2() # solve for DCT
      DFT_trunc, index = compress_block(DFT, K) # compress
      compressed[k:k+8, l:l+8] = DFT_trunc
  return compressed

def q_12(img, K):
  X_img_DCT = compress_img_DCT_2D(img, K)

  plt.imshow(np.log(1+np.abs(X_img_DCT)*100), cmap='gray')
  plt.title('Compressed DCT of the partitioned image')
  plt.colorbar()
  plt.show()

  X_img_DFT = compress_img_DFT_2D(img, K)

  plt.imshow(np.log(1+np.abs(X_img_DFT)*100), cmap='gray')
  plt.title('Compressed DFT of the partitioned image')

  plt.colorbar()
  plt.show()

  return X_img_DCT

if __name__ == '__main__':

    img = mpimg.imread('imgB_prenoise.png')

    plt.imshow(img, cmap='gray')
    plt.colorbar()
    plt.show()
    K_seq = [4,8,16,32]


    for i in np.arange(len(K_seq)):
        X_DCT = q_12(img, K_seq[i])


FileNotFoundError: ignored

# Section 1.3

In [ ]:
def table(quality):
  Q50 = [[  16,  11,  10,  16,  24,  40,  51,  61],
         [  12,  12,  14,  19,  26,  58,  60,  55],
         [  14,  13,  16,  24,  40,  57,  69,  56],
         [  14,  17,  22,  29,  51,  87,  80,  62],
         [  18,  22,  37,  56,  68, 109, 103,  77],
         [  24,  35,  55,  64,  81, 104, 113,  92],
         [  49,  64,  78,  87, 103, 121, 120, 101],
         [  72,  92,  95,  98, 112, 100, 103,  99]]
  if quality > 50:
    QX = (Q50*(np.ones((8,8))*((100 - quality) / 50))).round().astype(np.int32)
  else:
    QX = (Q50*(np.ones((8,8))*( 50 / quality))).round().astype(np.int32)
  return QX
  
def quantize(QX, block):
  return (block / QX).round().astype(np.int32)

def dequantize(QX, block):
  return (block * QX).round().astype(np.int32)

def q13(table):
  img = mpimg.imread('imgB_prenoise.png')
  N = img.shape[0]
  quantized = np.zeros([N, N], dtype=np.complex)
  for i in np.arange(0, N-7, 8):
    for j in np.arange(0, N-7, 8):
      subblock = img[i : i+8, j : j+8] * 256
      block_DCT = DCT_2D(subblock)
      block = block_DCT.solve2()
      quant_block = quantize(table, block)
      quantized[i : i+8, j : j+8] = quant_block
  return quantized

# Section 1.4 

In [ ]:
# calculate iDCT of each patch, then stitch them back together

def q14(X):
  N = X.shape[0]
  reconstructed_x = np.zeros([N, N], dtype=np.complex)

  for k in np.arange(0, N+1 - 8, 8):
    for l in np.arange(0, N+1 - 8, 8):
      # partition into separate 8x8 blocks
      block = X[k:k+8, l:l+8]
      iDCT_block = iDCT_2D(block)
      x_block = iDCT_block.solve()
      reconstructed_x[k:k+8, l:l+8] = x_block
  
  x_norm = abs(reconstructed_x) / np.amax(abs(reconstructed_x))

  plt.imshow(x_norm, cmap='gray')
  plt.colorbar()
  plt.show()

  return abs(reconstructed_x)

if __name__ == '__main__':
  img = mpimg.imread('imgB_prenoise.png')

  plt.imshow(img, cmap='gray')
  plt.colorbar()
  plt.show()
  K_list = [2, 4, 8, 16, 32]
  error = np.zeros(len(K_list))

  for i in np.arange(len(K_list)):
    DCT_X = q_12(img, K_list[i])
    x_K = q14(DCT_X)
    error[i] = np.linalg.norm(img - x_K, 2)    # reconstruction error

  plt.plot(K_list, error)
  plt.title('Reconstruction Error')
  plt.savefig('reconstructed_error.png')
  plt.xlabel('K (number of DFT coefficients)')
  plt.ylabel('Error')

In [ ]:
# for quantized DCT coefficients
def q14_quantized(QX, block):
  N = block.shape[0]
  dequantized_x = np.zeros([N, N], dtype=np.complex)
  for k in np.arange(0, N-7, 8):
    for l in np.arange(0, N-7, 8):
      X_block = block[k:k+8, l:l+8]
      X_block_dequantized = dequantize(QX, X_block)
      iDCT_block = iDCT_2D(X_block_dequantized)
      x_block = iDCT_block.solve()/256
      dequantized_x[k:k+8, l:l+8] = x_block
  energy_rec = np.amax(dequantized_x)

  # for reconstruction error
  x_hat = dequantized_x / energy_rec
  error = np.linalg.norm(block - x_hat)

  return dequantized_x / energy_rec

if __name__ == '__main__':
  img = mpimg.imread('imgB_prenoise.png')

  # quality = 50
  QX = table(50)
  X = q13(QX)
  x = q14_quantized(QX, X)
  plt.imshow(abs(x), cmap='gray')
  plt.title('Reconstructed image with standard quantization matrix')
  plt.savefig('Recons_jpeg.png')
  print(np.linalg.norm(img - abs(x), 2))
  plt.colorbar()
  plt.show()

  # quality = 95
  QX = table(95)
  X = q13(QX)
  x = q14_quantized(QX, X)
  plt.imshow(abs(x), cmap='gray')
  print(np.linalg.norm(img - abs(x), 2))

  plt.title('Reconstructed image with quantization matrix of value 95') 
  plt.savefig('Recons_jpeg.png')
  plt.colorbar()
  plt.show()

  # quality = 2
  QX = table(2)
  X = q13(QX)
  x = q14_quantized(QX, X)
  plt.imshow(abs(x), cmap='gray')
  print(np.linalg.norm(img - abs(x), 2))

  plt.title('Reconstructed image with quality = 2')
  plt.savefig('Reconshigh_jpeg.png')
  plt.colorbar()
  plt.show()